# CODIGOS PRINCIPALES CALIPEDIA-CALIBRANDO

Siempre que lo abras, ejecuta las importaciones!!!

In [ ]:
# Este codigo ------->   pip install pandas pyreadr numpy

In [ ]:
#Importaciones necesarias para que funcione el codigo
import pandas as pd
import pyreadr
import os
import numpy as np


Adicionalmente, para ejecutar correctamente este codigo, necesitamos tener los insumos (Base de datos en este caso "CaliBRANDO2024.dta") necesarios, para facilitar la comprension, estos insumos se cargaran en la carpeta "insumos" (Tranquilo si no la tienes el siguiente codigo la ejecutará).

Tambien, es necesario saber que se debe ejecutar en orden, ya que como en R, aqui usaremos variables, listas, dataFrames, etc... , que se usaran mas adelante. 

In [ ]:
# Nombre de la carpeta donde antes de ejecutar el script se guardan los datos CaliBRANDO2024.dta
carpeta = "insumos"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

carpeta = "insumos/data"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

Esta linea es unicamente para cargar la base de datos, le llamamos df.

In [ ]:
df = pd.read_stata("insumos/data/Panel 2023_ 1.dta")  # Leer el archivo .dta

In [ ]:
# Mostrar las primeras filas del DataFrame
print(df.head())

In [ ]:
df

In [ ]:
shape = df.shape  # Obtener la forma del DataFrame  
print(shape)  # Mostrar la forma del DataFrame

## MENTAL HEALTH

### NOTA: 
Los siguientes codigos indican el procesamiento de las variables de interes para calcular indicadores de estres, depresion y ansiedad

* Estres: P10000801 P10000802 P10000803 P10000804 P10000805 P10000806 P10000807 P10000808 P10000809 P100008010
* Ansiedad: P10000901 P10000902 P10000903 P10000904 P10000905 P10000906 P10000907
* Depresion: P10001001 P10001002 P10001003 P10001004 P10001005 P10001006 P10001007 P10001008 P10001009

### Recode Estres: Estres autopercibido

In [ ]:
cols_categoricas=["P10000801", "P10000802", "P10000803", "P10000804", "P10000805", "P10000806", "P10000807", "P10000808", "P10000809", "P10000810"]
cols_invertir = ["P10000804", "P10000805", "P10000807", "P10000808"]

In [ ]:
print(df[cols_categoricas].dtypes)

In [ ]:
for col in cols_categoricas:
    print(f"{col}: {df[col].unique()}")

In [ ]:
# Mapeo de categorías a valores numéricos
mapa_categorias = {
    "Nunca": 0,
    "Casi Nunca": 1,
    "De vez en cuando": 2,
    "A menudo": 3,
    "Muy amenudo": 4,
    "No": 0,
    "Si": 1
}

# Convertir categorías a valores numéricos
for col in cols_categoricas:
    df[col] = df[col].map(mapa_categorias).astype("float64")

# Función corregida para invertir la escala (de 0 a 4)
def invertir_escala(valor):
    if pd.notna(valor):  # Verifica que el valor no sea NaN
        return 4 - valor  # Invierte la escala correctamente
    return valor  # Si es NaN, lo deja igual

# Aplicar la función a las columnas invertidas
for col in cols_invertir:
    df[f"in_{col}"] = df[col].apply(invertir_escala)


In [ ]:
df["estresado"] = (
    df[["P10000801", "P10000802", "P10000803", "in_P10000804",
        "in_P10000805", "P10000806", "in_P10000807", "in_P10000808",
        "P10000809", "P10000810"]]
    .fillna(0)  # Reemplaza NaN con 0
    .sum(axis=1)  # Suma fila por fila
)


In [ ]:
# Calcular frecuencia y porcentaje
frecuencia = df["in_P10000808"].value_counts(dropna=False)  
porcentaje = df["in_P10000808"].value_counts(normalize=True, dropna=False) * 100  

# Crear DataFrame
tabla_frecuencia = pd.DataFrame({
    "Frecuencia": frecuencia,
    "Porcentaje": porcentaje
})

# Agregar columna de frecuencia acumulada
tabla_frecuencia["Frecuencia Acumulada"] = tabla_frecuencia["Frecuencia"].cumsum()
tabla_frecuencia["Porcentaje Acumulado"] = tabla_frecuencia["Porcentaje"].cumsum()

# Agregar fila de totales
total_frecuencia = tabla_frecuencia["Frecuencia"].sum()
total_porcentaje = tabla_frecuencia["Porcentaje"].sum()
tabla_frecuencia.loc["Total"] = [total_frecuencia, total_porcentaje, None, None]

print(tabla_frecuencia)


In [ ]:
# Calcular frecuencia absoluta
frecuencia = df["in_P10000808"].value_counts(dropna=False).sort_index()

# Calcular total sin incluir NaN
total_sin_na = df["in_P10000808"].count()

# Calcular porcentajes correctamente
porcentaje = (frecuencia / total_sin_na) * 100

# Crear DataFrame
tabla_frecuencia = pd.DataFrame({
    "Freq.": frecuencia,
    "Percent": porcentaje
})

# Agregar columna acumulada
tabla_frecuencia["Cum."] = tabla_frecuencia["Percent"].cumsum()

# Agregar fila de totales
tabla_frecuencia.loc["Total"] = [frecuencia.sum(), 100, None]

# Redondear como en Stata
tabla_frecuencia = tabla_frecuencia.round(2)

print(tabla_frecuencia)


In [ ]:
print(df["in_P10000808"].describe())

In [ ]:
print(df["estresado"].describe())

In [ ]:
# Clasificar en categoría de estrés (1 = estresado, 0 = no estresado)
df["cat_estres"] = df["estresado"].apply(lambda x: 1 if x >= 15 else 0)

# Contar cuántas personas hay en cada categoría
print(df["cat_estres"].value_counts())

# Mostrar en porcentaje
print(df["cat_estres"].value_counts(normalize=True) * 100)

### Balance afectivo

Se refiere a la diferencia entre la felicidad percibida del
individuo y su preocupacion. Esta variable fue normalizada entre 0 y 1 para compararse
con la medida de satisfacción con la vida y mantener una mejor interpretacion del balance
afectivo

In [ ]:
print(df["P03001501"].cat.categories)
print(df["P03000601"].cat.categories)


In [ ]:
# Mapeo de valores categóricos a números
mapeo = {
    "En ningún momento": 0,
    "Todo el tiempo": 10
}

# Reemplazar y convertir a float
df["P03001501"] = df["P03001501"].replace(mapeo).astype(float)
df["P03000601"] = df["P03000601"].replace(mapeo).astype(float)

# Verificar que la conversión fue exitosa
print(df[["P03001501", "P03000601"]].dtypes)  # Ambas deben ser float
print(df[["P03001501", "P03000601"]].head())  # Verificar valores

In [ ]:
df["balance_afectivo"] = df["P03001501"] - df["P03000601"]

In [ ]:
df["balance_afectivo"] = (df["balance_afectivo"] - df["balance_afectivo"].min()) / (df["balance_afectivo"].max() - df["balance_afectivo"].min())

In [ ]:
print(df["balance_afectivo"].describe())  # Para ver estadísticas
print(df["balance_afectivo"].head())  # Para ver los primeros valores

### Recode Ansiedad: Ansiedad autopercibida

In [ ]:
cols_categoricas=["P10000901", "P10000902", "P10000903", "P10000904", "P10000905", "P10000906", "P10000907"]
for col in cols_categoricas:
    print(f"{col}: {df[col].unique()}")

In [ ]:
# Mapeo de categorías a valores numéricos
mapa_categorias = {
    "Ningun día": 0,
    "Varios días": 1,
    "Más de la mitad de los días": 2,
    "Casi todos los días": 3,
}

# Convertir categorías a valores numéricos
for col in cols_categoricas:
    df[col] = df[col].map(mapa_categorias).astype("float64")

In [ ]:
#Para ver tab de frecuencia de los codigos PXXXXXXXXXX
# Contar valores, incluyendo NaN
frecuencia = df["P10000902"].value_counts(dropna=False).sort_index()

# Calcular porcentaje
porcentaje = (frecuencia / frecuencia.sum()) * 100

# Calcular frecuencia acumulada
frecuencia_acumulada = frecuencia.cumsum()

# Calcular porcentaje acumulado
porcentaje_acumulado = porcentaje.cumsum()

# Crear DataFrame con los resultados
tabla_frecuencia = pd.DataFrame({
    "Frecuencia": frecuencia,
    "Porcentaje": porcentaje,
    "Frecuencia Acumulada": frecuencia_acumulada,
    "Porcentaje Acumulado": porcentaje_acumulado
})

# Agregar fila de total
tabla_frecuencia.loc["Total"] = [
    frecuencia.sum(), 100, None, None
]

# Mostrar tabla
print(tabla_frecuencia)


In [ ]:
# Asegurar que las columnas sean numéricas antes de sumar
cols_ansiedad = ["P10000901", "P10000902", "P10000903", "P10000904", "P10000905", "P10000906", "P10000907"]
df[cols_ansiedad] = df[cols_ansiedad].apply(pd.to_numeric, errors="coerce")  # Convierte a numérico, forzando NaN en errores

# Sumar las columnas de ansiedad
df["ansioso"] = df[cols_ansiedad].sum(axis=1, skipna=False)  # Respeta los NaN en la suma

# Crear la variable categórica cat_ansioso sin modificar NaN
df["cat_ansioso"] = pd.NA
df.loc[df["ansioso"].notna() & (df["ansioso"] < 5), "cat_ansioso"] = 0
df.loc[df["ansioso"].notna() & (df["ansioso"] >= 5), "cat_ansioso"] = 1

# Etiquetas para la variable categórica
ansiedad_labels = {0: "Menor nivel ansiedad", 1: "Mayor nivel ansiedad"}
df["cat_ansioso_label"] = df["cat_ansioso"].map(ansiedad_labels)

# Mostrar la tabla de frecuencia
tabla_frecuencia_ansioso = df["cat_ansioso_label"].value_counts(dropna=False).to_frame(name="Frecuencia")
tabla_frecuencia_ansioso["Porcentaje"] = (tabla_frecuencia_ansioso["Frecuencia"] / tabla_frecuencia_ansioso["Frecuencia"].sum()) * 100
tabla_frecuencia_ansioso["Frecuencia Acumulada"] = tabla_frecuencia_ansioso["Frecuencia"].cumsum()
tabla_frecuencia_ansioso["Porcentaje Acumulado"] = tabla_frecuencia_ansioso["Porcentaje"].cumsum()

# Agregar fila de totales
total_frecuencia = tabla_frecuencia_ansioso["Frecuencia"].sum()
total_porcentaje = tabla_frecuencia_ansioso["Porcentaje"].sum()
tabla_frecuencia_ansioso.loc["Total"] = [total_frecuencia, total_porcentaje, None, None]

print(tabla_frecuencia_ansioso)


### Recode Depresion: Depresion autopercibida

In [ ]:
cols_categoricas=["P10001001", "P10001002", "P10001003", "P10001004", "P10001005", "P10001006", "P10001007", "P10001008", "P10001009"]
for col in cols_categoricas:
    print(f"{col}: {df[col].unique()}")

In [ ]:
# Mapeo de categorías a valores numéricos
mapa_categorias = {
    "Ningun día": 0,
    "Varios días": 1,
    "Más de la mitad de los días": 2,
    "Casi todos los días": 3,
}

# Convertir categorías a valores numéricos
for col in cols_categoricas:
    df[col] = df[col].map(mapa_categorias).astype("float64")

In [ ]:
#Para ver tab de frecuencia de los codigos PXXXXXXXXXX
# Contar valores, incluyendo NaN
frecuencia = df["P10001001"].value_counts(dropna=False).sort_index()

# Calcular porcentaje
porcentaje = (frecuencia / frecuencia.sum()) * 100

# Calcular frecuencia acumulada
frecuencia_acumulada = frecuencia.cumsum()

# Calcular porcentaje acumulado
porcentaje_acumulado = porcentaje.cumsum()

# Crear DataFrame con los resultados
tabla_frecuencia = pd.DataFrame({
    "Frecuencia": frecuencia,
    "Porcentaje": porcentaje,
    "Frecuencia Acumulada": frecuencia_acumulada,
    "Porcentaje Acumulado": porcentaje_acumulado
})

# Agregar fila de total
tabla_frecuencia.loc["Total"] = [
    frecuencia.sum(), 100, None, None
]

# Mostrar tabla
print(tabla_frecuencia)


In [ ]:
# Definir las columnas para la variable "deprimido"
cols_depresion = ["P10001001", "P10001002", "P10001003", "P10001004", 
                  "P10001005", "P10001006", "P10001007", "P10001008", "P10001009"]

# Asegurar que las columnas sean numéricas antes de sumar
df[cols_depresion] = df[cols_depresion].apply(pd.to_numeric, errors="coerce")  # Convierte a numérico, forzando NaN en errores

# Sumar las columnas de depresión
df["deprimido"] = df[cols_depresion].sum(axis=1, skipna=False)  # Respeta los NaN en la suma

# Crear la variable categórica cat_depresion sin modificar NaN
df["cat_depresion"] = pd.NA
df.loc[df["deprimido"].notna() & (df["deprimido"] < 7), "cat_depresion"] = 0
df.loc[df["deprimido"].notna() & (df["deprimido"] >= 7), "cat_depresion"] = 1

# Etiquetas para la variable categórica
depresion_labels = {0: "Menor nivel depresion", 1: "Mayor nivel depresion"}
df["cat_depresion_label"] = df["cat_depresion"].map(depresion_labels)

# Crear la tabla de frecuencia
tabla_frecuencia_depresion = df["cat_depresion_label"].value_counts(dropna=False).to_frame(name="Frecuencia")
tabla_frecuencia_depresion["Porcentaje"] = (tabla_frecuencia_depresion["Frecuencia"] / tabla_frecuencia_depresion["Frecuencia"].sum()) * 100
tabla_frecuencia_depresion["Frecuencia Acumulada"] = tabla_frecuencia_depresion["Frecuencia"].cumsum()
tabla_frecuencia_depresion["Porcentaje Acumulado"] = tabla_frecuencia_depresion["Porcentaje"].cumsum()

# Agregar fila de totales
total_frecuencia = tabla_frecuencia_depresion["Frecuencia"].sum()
total_porcentaje = tabla_frecuencia_depresion["Porcentaje"].sum()
tabla_frecuencia_depresion.loc["Total"] = [total_frecuencia, total_porcentaje, None, None]

# Mostrar la tabla de frecuencia
print(tabla_frecuencia_depresion)


## SALUD FISICA

### NOTA: 
Revisar estos codigos usando el Marco Instrumental de CaliBRANDO

saludfisica = $P10000302$

saludmental = $P10000402$

altura = $P10000601$

peso = $P10000602$

In [ ]:
# Dummies de salud física
df['dsaludfisica7'] = (df['P10000302'] >= 7).astype(int)
df['dsaludfisica14'] = (df['P10000302'] >= 14).astype(int)

# Dummies de salud mental
df['dsaludmental7'] = (df['P10000402'] >= 7).astype(int)
df['dsaludmental14'] = (df['P10000402'] >= 14).astype(int)

# IMC
df['altura2'] = df['P10000601'] ** 2
df['imc2'] = df['P10000602'] / df['altura2']

# Clasificación del IMC sin modificar NaN
df['imc'] = pd.cut(df['imc2'], bins=[-float('inf'), 18.5, 25, 30, 59], labels=[1, 2, 3, 4])

# Convertir a int solo los valores no nulos
df['imc'] = df['imc'].astype('Int64')  # Usa 'Int64' para manejar NaN sin errores

# Etiquetas del IMC
imc_labels = {1: "Bajo peso", 2: "Normal", 3: "Sobrepeso", 4: "Obesidad"}
df['imc_label'] = df['imc'].map(imc_labels)


In [ ]:
# Calcular frecuencia absoluta
frecuencia = df["dsaludfisica7"].value_counts(dropna=False).sort_index()
# Calcular total sin incluir NaN
total_sin_na = df["dsaludfisica7"].count()
# Calcular porcentajes correctamente
porcentaje = (frecuencia / total_sin_na) * 100
# Crear DataFrame
tabla_frecuencia = pd.DataFrame({
    "Freq.": frecuencia,
    "Percent": porcentaje
})
# Agregar columna acumulada
tabla_frecuencia["Cum."] = tabla_frecuencia["Percent"].cumsum()
# Agregar fila de totales
tabla_frecuencia.loc["Total"] = [frecuencia.sum(), 100, None]
# Redondear como en Stata
tabla_frecuencia = tabla_frecuencia.round(2)
print(tabla_frecuencia)


In [ ]:
# Calcular frecuencia absoluta
frecuencia = df["dsaludfisica14"].value_counts(dropna=False).sort_index()
# Calcular total sin incluir NaN
total_sin_na = df["dsaludfisica14"].count()
# Calcular porcentajes correctamente
porcentaje = (frecuencia / total_sin_na) * 100
# Crear DataFrame
tabla_frecuencia = pd.DataFrame({
    "Freq.": frecuencia,
    "Percent": porcentaje
})
# Agregar columna acumulada
tabla_frecuencia["Cum."] = tabla_frecuencia["Percent"].cumsum()
# Agregar fila de totales
tabla_frecuencia.loc["Total"] = [frecuencia.sum(), 100, None]
# Redondear como en Stata
tabla_frecuencia = tabla_frecuencia.round(2)
print(tabla_frecuencia)


In [ ]:
# Calcular frecuencia y porcentaje con etiquetas en lugar de números
frecuencia = df["imc_label"].value_counts(dropna=False)  
porcentaje = df["imc_label"].value_counts(normalize=True, dropna=False) * 100  

# Crear DataFrame
tabla_frecuencia = pd.DataFrame({
    "Frecuencia": frecuencia,
    "Porcentaje": porcentaje
})

# Agregar columna de frecuencia acumulada
tabla_frecuencia["Frecuencia Acumulada"] = tabla_frecuencia["Frecuencia"].cumsum()
tabla_frecuencia["Porcentaje Acumulado"] = tabla_frecuencia["Porcentaje"].cumsum()

# Agregar fila de totales
total_frecuencia = tabla_frecuencia["Frecuencia"].sum()
total_porcentaje = tabla_frecuencia["Porcentaje"].sum()
tabla_frecuencia.loc["Total"] = [total_frecuencia, total_porcentaje, None, None]

print(tabla_frecuencia)


## INGRESOS

### NOTA: 
* Los siguientes codigos indican el procesamiento de la variable de ingresos que es categorica, para volverla continua. Se presenta por anio
* Este codigo fue creado por Eduardo LORA
* Para alicar este codigo por anio, es necesario revisar los salarios minimos de cada anio. Para el 2024, se debe pensar si extender el limite superior

In [ ]:
# Diccionario de mapeo para convertir ingresos a valores numéricos
ingresos_map = {
    "No tiene ingresos": np.nan,
    "Mas de 8 SMMLV": 5,
    "Menos de 1 SMMLV": 1,
    "Más de 1 SMMLV y menos de 2  SMMLV": 2,
    "Más de 2 SMMLV y menos de 4 SMMLV": 3,
    "Más de 4 SMMLV y menos de 8 SMMLV": 4,
    "No sabe ó no responde": np.nan  # En Stata este valor se excluye en los cálculos
}

# Aplicar el mapeo
df["promedioingresos1"] = df["P07000101"].map(ingresos_map)

# Generar la tabla de frecuencias
tabla_frecuencias = df["P07000101"].value_counts(dropna=False).reset_index()
tabla_frecuencias.columns = ["Promedio de ingresos", "Freq."]

# Calcular el porcentaje y acumulado
tabla_frecuencias["Percent"] = (tabla_frecuencias["Freq."] / tabla_frecuencias["Freq."].sum()) * 100
tabla_frecuencias["Cum."] = tabla_frecuencias["Percent"].cumsum()

# Mostrar la tabla
print(tabla_frecuencias)

In [ ]:
ingresoMax=10000000

# Definir los rangos mínimos y máximos de ingresos por año
rangos_ingresos = {
    2014: [0, 616027, 1232054, 2464108, 4928216, ingresoMax],
    2015: [0, 644350, 1288700, 2577400, 5154800, ingresoMax],
    2016: [0, 689454, 1378908, 2757816, 5515632, ingresoMax],
    2017: [0, 737717, 1475434, 2950868, 5901736, ingresoMax],
    2018: [0, 781242, 1562484, 3124968, 6249936, ingresoMax],
    2019: [0, 828211, 1656422, 3312844, 6625688, ingresoMax],
    2021: [0, 908526, 1817052, 3634104, 7268208, ingresoMax],
    2022: [0, 1160000, 2320000, 4640000, 9280000, ingresoMax]
}

for year in rangos_ingresos.keys():
    # Variable aleatoria uniforme entre 0 y 1
    df[f'ingc{year}'] = np.where(df['P01000801'] == year, np.random.uniform(size=len(df)), np.nan)

    # Obtener el rango de ingresos
    df[f'rango_ing{year}'] = np.where(df['P01000801'] == year, df['promedioingresos1'], np.nan)

    # Asignar valores mínimos y máximos según el rango de ingresos
    df[f'xmin{year}'] = df[f'rango_ing{year}'].map(
        lambda x: rangos_ingresos[year][int(x) - 1] if pd.notnull(x) and 1 <= x <= 6 else np.nan
    )

    df[f'xmax{year}'] = df[f'rango_ing{year}'].map(
        lambda x: rangos_ingresos[year][int(x)] if pd.notnull(x) and 1 <= x <= 5 else 10000000
    )

    # Calcular el ingreso basado en xmin y xmax
    df[f'ingreso{year}'] = df[f'xmin{year}'] + (df[f'xmax{year}'] - df[f'xmin{year}']) * df[f'ingc{year}']
    
    # Logaritmo del ingreso
    df[f'lningreso{year}'] = np.log(df[f'ingreso{year}'])


In [ ]:
# Suponiendo que tienes un DataFrame llamado df con las columnas mencionadas
df['ingresopanel'] = None  # Crear la columna inicialmente

# Asignar valores según el año en la columna 'P01000801'
for year in [2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022]:
    df.loc[df['P01000801'] == year, 'ingresopanel'] = df[f'ingreso{year}']

# Convertir la columna a tipo numérico (por si quedan valores nulos o de otro tipo)
df['ingresopanel'] = pd.to_numeric(df['ingresopanel'], errors='coerce')

# Obtener la media de 'ingresopanel' agrupado por 'P01000801'
result = df.groupby('P01000801')['ingresopanel'].mean().reset_index()

# Mostrar el resultado
print(result)


## FORMAL/INFORMAL

### NOTA:
Los siguientes codigos indican el procesamiento de las variables de interes para calcular indicadores de estres, depresion y ansiedad

//Informal employment: Non- contribution to health and pension system

* salud y pension: $P06000701$

* ninguno: $P06000703$

* solo salud: $P06000704$

* solo pension: $P06000705$

In [ ]:
cols_categoricas=['P06000701', 'P06000703', 'P06000704', 'P06000705']
print(df[cols_categoricas].dtypes)

In [ ]:
for col in cols_categoricas:
    print(f"{col}: {df[col].unique()}")

In [ ]:
# Mapeo de categorías a valores numéricos
mapa_categorias = {
    "No": 0,
    "Si": 1
}

# Convertir categorías a valores numéricos
for col in cols_categoricas:
    df[col] = df[col].map(mapa_categorias).astype("float64")

In [ ]:
# Convertir a numérico si es necesario
for col in ['P06000701', 'P06000703', 'P06000704', 'P06000705']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Inicializar formalidad con NaN
df['formalidad'] = None  

# Asignar valores según las condiciones
df.loc[df['P06000701'] == 1, 'formalidad'] = 1
df.loc[(df['P06000703'] == 1) | (df['P06000704'] == 1) | (df['P06000705'] == 1), 'formalidad'] = 0

# Convertir formalidad a numérico
df['formalidad'] = pd.to_numeric(df['formalidad'], errors='coerce')

# Verificar si hay valores NaN
print(df.isna().sum())

In [ ]:
# Mostrar tabla de frecuencia
frecuencia = df["formalidad"].value_counts(dropna=False)
porcentaje = df["formalidad"].value_counts(normalize=True, dropna=False) * 100

tabla_frecuencia = pd.DataFrame({
    "Frecuencia": frecuencia,
    "Porcentaje": porcentaje
})

tabla_frecuencia["Frecuencia Acumulada"] = tabla_frecuencia["Frecuencia"].cumsum()
tabla_frecuencia["Porcentaje Acumulado"] = tabla_frecuencia["Porcentaje"].cumsum()

# Total
total_frecuencia = tabla_frecuencia["Frecuencia"].sum()
total_porcentaje = tabla_frecuencia["Porcentaje"].sum()
tabla_frecuencia.loc["Total"] = [total_frecuencia, total_porcentaje, None, None]

print(tabla_frecuencia)

## MINORIA ETNICA


* etnia POLIS: $P02001201$ (desde 2014 a 2019)

* etnia DANE: $P02001202$ (desde 2021 en adelante)

In [ ]:
cols_categoricas=['P02001201', 'P02001202']
print(df[cols_categoricas].dtypes)

In [ ]:
for col in cols_categoricas:
    print(f"{col}: {df[col].unique()}")

In [ ]:
# Valores de el Archivo Stata, mal puestos!!!!
mapa_p02001201 = {
    'Indígena': 0,
    'Gitano': 0,
    'Raizal': 1,
    'Palenquero': 1,
    'Negro ó afro': 0,
    'Ninguno': None
}

mapa_p02001202 = {
    'Blanco': 1,
    'Mestizo': 1,
    'Indigena': 1,
    'Negro ó afro': 1,
    'Otro': 1,
    'Ninguno': 0
}

In [ ]:
# Como creo que se correguiria Segun las categorias!!
mapa_p02001201 = {
    'Indígena': 1,
    'Gitano': 1,
    'Raizal': 1,
    'Palenquero': 1,
    'Negro ó afro': 1,
    'Ninguno': None
}

mapa_p02001202 = {
    'Blanco': 0,
    'Mestizo': 0,
    'Indigena': 1,
    'Negro ó afro': 1,
    'Otro': 0,
    'Ninguno': None
}

In [ ]:
# Inicializar la columna 'minoriaetnica' con valores nulos
df['minoriaetnica'] = None

# Definir el mapeo de valores para P02001201 (2014-2019) y P02001202 (2021 en adelante)
mapa_p02001201 = {
    'Indígena': 1,
    'Gitano': 1,
    'Raizal': 1,
    'Palenquero': 1,
    'Negro ó afro': 1,
    'Ninguno': None
}

mapa_p02001202 = {
    'Blanco': 0,
    'Mestizo': 0,
    'Indigena': 1,
    'Negro ó afro': 1,
    'Otro': 0,
    'Ninguno': None
}

# Aplicar el mapeo a las columnas correspondientes
df['minoriaetnica'] = df['P02001201'].map(mapa_p02001201)

# Si existe P02001202, actualizar los valores en base a esta columna
df.loc[df['P02001202'].notna(), 'minoriaetnica'] = df['P02001202'].map(mapa_p02001202)

# Convertir la columna a tipo numérico
df['minoriaetnica'] = pd.to_numeric(df['minoriaetnica'], errors='coerce')

# Mostrar tabla de frecuencia
frecuencia = df["minoriaetnica"].value_counts(dropna=False)  
porcentaje = df["minoriaetnica"].value_counts(normalize=True, dropna=False) * 100  

# Crear DataFrame de resumen
tabla_frecuencia = pd.DataFrame({
    "Frecuencia": frecuencia,
    "Porcentaje": porcentaje
})

# Agregar columna de frecuencia acumulada
tabla_frecuencia["Frecuencia Acumulada"] = tabla_frecuencia["Frecuencia"].cumsum()
tabla_frecuencia["Porcentaje Acumulado"] = tabla_frecuencia["Porcentaje"].cumsum()

# Agregar fila de totales
total_frecuencia = tabla_frecuencia["Frecuencia"].sum()
total_porcentaje = tabla_frecuencia["Porcentaje"].sum()
tabla_frecuencia.loc["Total"] = [total_frecuencia, total_porcentaje, None, None]

# Renombrar valores para que coincidan con Stata
tabla_frecuencia.rename(index={0.0: "No minoria", 1.0: "Minoria"}, inplace=True)

# Mostrar resultados
print(tabla_frecuencia)


## NSE

In [ ]:
# Crear la nueva columna NSE a partir de P02000801
df['NSE'] = df['P02000801']

# Asignar valores según las condiciones
df.loc[df['P02000801'].isin([1, 2]), 'NSE'] = 1  # NSE Bajo (estrato 1 y 2)
df.loc[df['P02000801'].isin([3, 4]), 'NSE'] = 2  # NSE Medio (estrato 3 y 4)
df.loc[df['P02000801'].isin([5, 6]), 'NSE'] = 3  # NSE Alto (estrato 5 y 6)

# Convertir a tipo numérico
df['NSE'] = pd.to_numeric(df['NSE'], errors='coerce')

# Etiquetas para los niveles socioeconómicos
nse_labels = {1: "NSE Bajo", 2: "NSE Medio", 3: "NSE Alto"}

# Calcular frecuencias y porcentajes
frecuencia = df["NSE"].value_counts(dropna=False).sort_index()
porcentaje = df["NSE"].value_counts(normalize=True, dropna=False).sort_index() * 100

# Crear DataFrame de resumen
tabla_frecuencia = pd.DataFrame({
    "Frecuencia": frecuencia,
    "Porcentaje": porcentaje
})

# Agregar columna de frecuencia acumulada
tabla_frecuencia["Frecuencia Acumulada"] = tabla_frecuencia["Frecuencia"].cumsum()
tabla_frecuencia["Porcentaje Acumulado"] = tabla_frecuencia["Porcentaje"].cumsum()

# Agregar fila de totales
total_frecuencia = tabla_frecuencia["Frecuencia"].sum()
total_porcentaje = tabla_frecuencia["Porcentaje"].sum()
tabla_frecuencia.loc["Total"] = [total_frecuencia, total_porcentaje, None, None]

# Renombrar los índices con etiquetas
tabla_frecuencia.rename(index=nse_labels, inplace=True)

# Mostrar resultados
print(tabla_frecuencia)


In [ ]:
df

# Final

In [ ]:
df.to_csv("insumos/data/data_&_variables_nuevas_Calipedia.csv", index=False, encoding="utf-8", sep=";")
#df.to_excel("insumos/datos_con_scripts_Calipedia.xlsx", index=False, engine="openpyxl")
